# TFX - Deploy Pipeline to AI Platform Managed Pipelines

The purpose of this notebook is to compile and run the TFX pipeline to AI Platform Managed Pipelines. The notebook covers the following tasks:
1. Set pipeline configurations
2. Build Container Image
3. Compile TFX Pipeline
4. Submit a pipeline job to AI Platform Pipelines (Managed).

## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import tensorflow as tf

In [ ]:
PROJECT = 'ksalama-cloudml' # Change to your project Id.
REGION = 'us-central1'
BUCKET = 'ksalama-cloudml-us' # Change to your bucket.
API_KEY = '' # Change to your API Key.

In [ ]:
os.environ["API_KEY"] = API_KEY
os.environ["DATASET_DISPLAYNAME"] = 'chicago_taxi_tips'
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION
os.environ["GCS_LOCATION"] = f"gs://{BUCKET}/ucaip_demo/chicago_taxi/pipelines_managed_runner"
os.environ["TRAIN_LIMIT"] = "85000"
os.environ["TEST_LIMIT"] = "15000"
os.environ["BEAM_RUNNER"] = "DataflowRunner"
os.environ["TRAINING_RUNNER"] = "caip"

In [ ]:
from tfx_pipeline import config
for key, value in config.__dict__.items():
    if key.isupper(): print(f'{key}: {value}')

## Build Container Image

In [ ]:
!echo $IMAGE_URI

In [ ]:
!gcloud builds submit --tag $IMAGE_URI . --timeout=15m

## Compile Pipeline

In [ ]:
from tfx_pipeline import managed_runner

In [ ]:
pipeline_definition_file = managed_runner.compile_pipeline()

## Submit Run to AI Platform Managed Pipelines

In [ ]:
from aiplatform.pipelines import client

pipeline_client = client.Client(
    project_id=PROJECT,
    region=REGION,
    api_key=API_KEY
)

pipeline_client.create_run_from_job_spec(
    job_spec_path=pipeline_definition_file,
    parameter_values={
        'learning_rate': 0.003,
        'batch_size': 512,
        'hidden_units': '128,128'
        'num_epchs': 50,
    }
)

![Pipeline execution](managed-pipeline.png)